In [ ]:
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
from torchvision.transforms import v2
from torch.utils.data import DataLoader

In [42]:
SET_ROOT = './data'
NUM_CHANNELS = 3 # RGB format
NUM_CLASSES_CIFAR10 = 10 # In CIFAR10 Dataset
EPOCHS = 20 #70
BATCH_SIZE = 1000 # Required = 1, for now
ALLOW_CUDA_TF32_CORES = True # Enable tensor cores on nvidia gpus (ampere gen. and above)
ALLOW_CUDA = True # Overwrites all CUDA setup; enables standard CPU usage. 

In [45]:
def loadData():
    transf_test = v2.Compose([
        v2.RandomHorizontalFlip(),
        v2.RandomVerticalFlip(),
        v2.ToTensor()
    ])
    transf_train = v2.Compose([
        v2.ToTensor()
    ])

    testset = CIFAR10(root=SET_ROOT, 
                    train=False, 
                    download=True, 
                    transform=transf_test)

    trainset = CIFAR10(root=SET_ROOT, 
                    train=True, 
                    download=True, 
                    transform=transf_train)

    print(f"Len test: {len(testset)}, Len train: {len(trainset)} using BATCH_SIZE={BATCH_SIZE}")
    train_size= int(0.8 * len(trainset))
    val_size = len(trainset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(trainset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, val_loader, test_loader

In [44]:
# CUDA setup

if ALLOW_CUDA:
    cuda_available = torch.cuda.is_available()
    if not cuda_available:
        device = torch.device("cpu")
        print(device)

    else:
        device = torch.device("cuda")
        print(device, "devices: ", torch.cuda.device_count())
        print(torch.cuda.current_device(), ", name:", torch.cuda.get_device_name(torch.cuda.current_device()))

        # For my home GPU, for matrix multiplication speedup
        # If you dont have a nvidia 30-series or above disable 
        # via the ALLOW_CUDA_TF32_CORES in the top of this file
        # -Hannes
        if ALLOW_CUDA_TF32_CORES:
            print("Enabling TF32 matmul")
            torch.backends.cuda.matmul.allow_tf32 = True
            torch.backends.cudnn.allow_tf32 = True
        else:
            # Revert to default
            torch.backends.cuda.matmul.allow_tf32 = False
            torch.backends.cudnn.allow_tf32 = False
else:
    device = torch.device("cpu")
    print(device)

cuda devices:  1
0 , name: NVIDIA GeForce RTX 3070
Enabling TF32 matmul


In [46]:
class CNN1(nn.Module):
    def __init__(self, act_func=nn.LeakyReLU):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=NUM_CHANNELS, out_channels=10, kernel_size=2, device=device)
        self.af1 = act_func()

        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=2, device=device)
        self.af2 = act_func()

        self.pool1 = nn.MaxPool2d(kernel_size=2)

        # In-features is now hardcoded
        # Figure out how to calculate dynamically
        self.fc1 = nn.Linear(in_features=4500, out_features=NUM_CLASSES_CIFAR10, device=device)
        self.lsfm1 = nn.LogSoftmax(dim=0)

    def forward(self, x):
        x = self.conv1(x)
        x = self.af1(x)
        x = self.conv2(x)
        x = self.af2(x)
        x = self.pool1(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = self.lsfm1(x)
        return x

    def train_model(self, criterion, optimizer, train_loader):
        for epoch in range(EPOCHS):
            self.train() # training mode       
            for batch_i, (images, labels) in enumerate(train_loader):
                images, labels = images.float().to(device), labels.to(device) 
                loss = criterion(self(images), labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            if epoch % 1 == 0:
                print(epoch, loss.item(), batch_i+1)


    def test_model(self, test_loader):
        with torch.no_grad():
            self.eval() # evaluation mode
            correct, total = 0,0 
            for batch_i, (images, labels) in enumerate(test_loader):
                images, labels = images.float().to(device), labels.to(device)
                pred = torch.argmax(self(images), dim=1)
                correct += (pred == labels).sum()
                total += len(pred)

            print(f"Correct: {correct}, Total: {total}, Acc: {correct/total*100}%")

In [47]:
train_loader, val_loader, test_loader = loadData()

C:\Users\hanne\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\transforms\v2\_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


Files already downloaded and verified
Files already downloaded and verified
Len test: 10000, Len train: 50000 using BATCH_SIZE=1000


In [48]:
# Default: LeakyReLU, SGD
model = CNN1()
if torch.cuda.is_available() and ALLOW_CUDA: 
    print("Using cuda")
    model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
model.train_model(criterion, optimizer, train_loader)
model.test_model(test_loader)

Using cuda
0 2.3004112243652344 40
1 2.2993569374084473 40
2 2.3000519275665283 40
3 2.299765110015869 40
4 2.299621343612671 40
5 2.29911208152771 40
6 2.299020767211914 40
7 2.29963755607605 40
8 2.298851490020752 40
9 2.298555612564087 40
10 2.2984063625335693 40
11 2.298401355743408 40
12 2.2969510555267334 40
13 2.296391010284424 40
14 2.29536771774292 40


KeyboardInterrupt: 

In [ ]:
# LeakyReLU, Adam
model2 = CNN1()
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.0001)
criterion2 = nn.CrossEntropyLoss()
model2.train_model(criterion2, optimizer2)
model2.test_model()

In [ ]:
# Tanh, SGD
model3 = CNN1(act_func=nn.Tanh)
optimizer3 = torch.optim.SGD(model3.parameters(), lr=0.0001)
criterion3 = nn.CrossEntropyLoss()
model3.train_model(criterion3, optimizer3)
model3.test_model()